# 수집할때마다 중간저장- 상세정보 및 xpath 다시 점검하기(개선 필요)

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os

# WebDriver 실행
driver = webdriver.Chrome()
url = "https://www.saramin.co.kr/zf_user/search/recruit?searchType=search&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&keydownAccess=&searchword=%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D%EA%B0%80&panel_type=&search_optional_item=y&search_done=y&panel_count=y&preview=y&recruitPage=16&recruitSort=relation&recruitPageCount=40&inner_com_type=&show_applied=&quick_apply=&except_read=&ai_head_hunting=&mainSearch=n"
driver.get(url)

print("🔹 사람인 사이트 접속 완료. 페이지가 로드될 때까지 기다립니다...")
time.sleep(3)  # 페이지 로딩 대기

csv_filename = "사람인_채용정보_최종_데이터분석가_4.csv"
collected_ids = set()  # 중복 방지를 위한 ID 저장
qualification_list = ['정보처리기사', '빅데이터분석기사', 'ADSP', 'ADP', 'SQLD', 'SQLP']
language_list = ['파이썬', 'Python', 'python','SQL', 'MySQL', 'sql', 'mysql', 'Java', 'java', 'C', 'c']

def collect_data(max_jobs=40):
    """현재 페이지의 채용 공고 정보를 수집하여 CSV 파일에 저장"""
    global collected_ids
    
    for row_idx in range(1, max_jobs + 1):  # 첫 페이지는 41개, 이후는 40개
        try:
            title_xpath = f'//*[@id="recruit_info_list"]/div[1]/div[{row_idx}]/div[2]/h2/a/span'
            title_elements = driver.find_elements(By.XPATH, title_xpath)
            
            if not title_elements:
                break  # 공고가 없으면 종료
            
            title_element = title_elements[0]
            print(title_element)
            title = title_element.text.strip()
            
            # 중복 체크
            if title in collected_ids:
                continue  

            # 현재 창 저장
            main_window = driver.current_window_handle

            # 공고 URL 저장
            job_url = title_element.find_element(By.XPATH, "..").get_attribute("href")

            # 공고 클릭 (새 창에서 열림)
            title_element.click()
            time.sleep(2)  # 상세 페이지 로딩 대기

            # 새 창 탐색
            WebDriverWait(driver, 5).until(EC.number_of_windows_to_be(2))
            new_window = [window for window in driver.window_handles if window != main_window][0]
            driver.switch_to.window(new_window)
            
            # 상세 정보 크롤링
            try:
                # 기업명
                company_name = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/section[1]/div[1]/div[1]/div/div[1]/a[1]').text.strip()
            except:
                company_name = "N/A"

            try:
                # 경력
                experience_type = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/section[1]/div[1]/div[2]/div/div[1]/dl[1]/dd').text.strip()
            except:
                experience_type = "N/A"

            try:
                # 학력
                education_type = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/section[1]/div[1]/div[2]/div/div[1]/dl[2]/dd').text.strip()
            except:
                education_type = "N/A"

            try:
                # 근무 형태
                job_type = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/section[1]/div[1]/div[2]/div/div[1]/dl[3]/dd/strong').text.strip()
            except:
                job_type = "N/A"

            try:
                # 근무 지역
                location = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/section[1]/div[1]/div[2]/div/div[2]/dl[3]/dd').text.strip()
            except:
                location = "N/A"

            try:
                # 급여
                salary_info = driver.find_element(By.XPATH, '//*[@id="content"]/div[3]/section[1]/div[1]/div[2]/div/div[2]/dl[1]/dd').text.strip()
            except:
                salary_info = "N/A"

            try:
                job_details = driver.find_element(By.XPATH, '/html/body').text.strip()


                # 자격증과 언어를 찾기 위한 변수
                qualification = "N/A"
                language = "N/A"

                # 자격증 체크
                for q in qualification_list:
                    if q in job_details:
                        qualification = q
                        break  # 첫 번째 자격증 찾으면 종료

                # 언어 체크
                for lang in language_list:
                    if lang in job_details:
                        language = lang
                        break  # 첫 번째 언어 찾으면 종료


            except:
                job_detail = "N/A"

            # 데이터 저장 (매번 즉시 저장)
            df = pd.DataFrame([[title, company_name, job_type, location, salary_info, qualification, language, job_url]],
                              columns=["채용공고", "회사명", "직무", "근무지역", "급여정보", "자격증", "언어", "공고URL"])
            df.to_csv(csv_filename, index=False, encoding="utf-8-sig", mode='a', header=not os.path.exists(csv_filename))
            print(f"✅ 저장 완료: {title}")

            # 중복 방지를 위해 ID 저장
            collected_ids.add(title)

            # 상세 페이지 닫기
            driver.close()
            
            # 원래 창으로 복귀
            driver.switch_to.window(main_window)
            time.sleep(2)

        except Exception as e:
            print(f"⚠️ 오류 발생: {e}")
            driver.switch_to.window(main_window)
            continue

# 페이지 크롤링 로직
def scrape_pages():
    """모든 페이지를 탐색하며 데이터 수집"""
    collect_data(41)  # 첫 페이지는 41개

    for start_page in range(1, 51, 10):  # 1~10, 11~20, ..., 41~50 페이지 그룹별 크롤링
        for page in range(start_page + 1, start_page + 10):  # 2~10, 12~20, ..., 42~50
            try:
                page_xpath = f'//*[@id="recruit_info_list"]/div[2]/div/a[{page - start_page}]'
                page_buttons = driver.find_elements(By.XPATH, page_xpath)
                if page_buttons:
                    page_buttons[0].click()
                    time.sleep(2)
                    collect_data()
            except Exception as e:
                print(f"⚠️ 페이지 {page} 이동 중 오류 발생: {e}")

        try:
            next_button_xpath = '//*[@id="recruit_info_list"]/div[2]/div/a[10]'
            next_buttons = driver.find_elements(By.XPATH, next_button_xpath)
            if next_buttons:
                next_buttons[0].click()
                time.sleep(3)
                print(f"📌 다음 페이지 그룹 ({start_page+10}~{start_page+19}) 이동 완료.")
                collect_data()
            else:
                print("✅ 모든 페이지 크롤링 완료!")
                return
        except Exception as e:
            print(f"✅ 크롤링 종료! (오류: {e})")
            return

# 실행
scrape_pages()

# 종료
driver.quit()
print("✅ 최종 데이터 크롤링 완료! CSV 파일 확인:")
df_final = pd.read_csv(csv_filename)
print(df_final.head())


🔹 사람인 사이트 접속 완료. 페이지가 로드될 때까지 기다립니다...
<selenium.webdriver.remote.webelement.WebElement (session="aa5b249f2e6e2fe1f5af127630fd2782", element="f.6AF7BCE43CFCAE087D62E21EF22F5080.d.B59AB3CD12FD7A59DC9F43AF95C4228B.e.2485")>
✅ 저장 완료: 광전송 장비 기술지원 채용
<selenium.webdriver.remote.webelement.WebElement (session="aa5b249f2e6e2fe1f5af127630fd2782", element="f.6AF7BCE43CFCAE087D62E21EF22F5080.d.B59AB3CD12FD7A59DC9F43AF95C4228B.e.2535")>
✅ 저장 완료: [에이아이매틱스] 병역지정업체 전문연구요원
<selenium.webdriver.remote.webelement.WebElement (session="aa5b249f2e6e2fe1f5af127630fd2782", element="f.6AF7BCE43CFCAE087D62E21EF22F5080.d.B59AB3CD12FD7A59DC9F43AF95C4228B.e.2586")>
✅ 저장 완료: 2025년 빅데이터 분석 경력자 모집
<selenium.webdriver.remote.webelement.WebElement (session="aa5b249f2e6e2fe1f5af127630fd2782", element="f.6AF7BCE43CFCAE087D62E21EF22F5080.d.B59AB3CD12FD7A59DC9F43AF95C4228B.e.2635")>
✅ 저장 완료: 경력 직원 채용(데이터 처리가공 및 개발기술지원)
<selenium.webdriver.remote.webelement.WebElement (session="aa5b249f2e6e2fe1f5af127630fd2782", element="f.